In [1]:
import numpy as np
import numba
import scipy

In [2]:
from scipy.stats import truncnorm

In [3]:
import sympy
from sympy import *  # Symbolic mathematics
from sympy.vector import *
from sympy.physics.mechanics import *
from sympy import abc, pi as Pi, I as I

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
sympy.interactive.printing.init_printing(use_latex='mathjax')

In [6]:
np.set_printoptions(precision=5, threshold=6, edgeitems=3, linewidth=75, 
                    suppress=False, nanstr=None, infstr=None, 
                    formatter=None)

import pandas as pd

#  Variables

In [ ]:
E = ReferenceFrame('E')  # Vectors

In [ ]:
m = symbols('m', positive=True)  # Mass

In [ ]:
size = 2
phi = symbols('varphi_0:{}'.format(size), real=True)  # Angle [-pi, pi]
e = [cos(sym) * E.x + sin(sym) * E.y for sym in phi]  # Unit vectors
vphi = symbols('varphi', real=True)  # Angle [-pi, pi]

# Adjusting force

$\mathbf{f}^{adj} = \frac{m}{\tau^{adj}} (v_{0} \cdot \hat{\mathbf{e}} -\mathbf{v}) $

In [ ]:
tau, v0, v = symbols('tau v0 v', positive=True)
f_adj = m / tau * (v0 * e[0] -  v * e[1])

In [ ]:
f_adj.magnitude().simplify()

In [ ]:
_.subs(phi[0]-phi[1], vphi)

# Social Force

$$
\begin{split}a &= \tilde{\mathbf{v}} \cdot \tilde{\mathbf{v}} \\
b &= -\tilde{\mathbf{x}} \cdot \tilde{\mathbf{v}} \\
c &= \tilde{\mathbf{x}} \cdot \tilde{\mathbf{x}} - \tilde{r}^{2} \\
d &= \sqrt{b^{2} - a c} \\
\tau &= \frac{b - d}{a}.\end{split}
$$

$$
\begin{split}\mathbf{f}^{soc} &= -\nabla_{\tilde{\mathbf{x}}} E(\tau) \\
&= -\nabla_{\tilde{\mathbf{x}}} \left(\frac{k}{\tau^{2}} \exp \left( -\frac{\tau}{\tau_{0}} \right) \right) \\
&= - \left(\frac{k}{a \tau^{2}}\right) \left(\frac{2}{\tau} + \frac{1}{\tau_{0}}\right) \exp\left (-\frac{\tau}{\tau_{0}}\right ) \left(\tilde{\mathbf{v}} -\frac{a \tilde{\mathbf{x}} + b \tilde{\mathbf{v}}}{d} \right),\end{split}
$$

In [ ]:
x, v, r_tot = symbols('x, v, r_tot', positive=True)
x2, v2 = x * e[0], v * e[1]
a = dot(v2, v2).simplify()
b = -dot(x2, v2).simplify()
c = dot(x2, x2).simplify() - r_tot**2
d = sqrt(b**2 - a * c).simplify()

time-to-collision

In [ ]:
tau = simplify((b - d) / a)

In [ ]:
tau = tau.subs(phi[0]- phi[1], vphi)

In [ ]:
tau

tau when agents are in head-to-head collision trajectory

In [ ]:
tau_min = simplify(tau.subs(vphi, Pi))

In [ ]:
tau_min

In [ ]:
tau_min.subs({x: R - 2 * t * v_max, v: 2 * v_max, r_tot: 2 * r})

In [ ]:
simplify(_)

magnitude part of force

In [ ]:
tau, tau_0 = symbols('tau tau_0', positive=True)

In [ ]:
mag = -1/(a * tau**2) * (2 / tau + 1/ tau_0) * exp(-tau/tau_0)

In [ ]:
mag_max = mag.subs(tau, tau_min)

In [ ]:
mag_max

direction part of force

In [ ]:
drc = v2 - (a * x2 + b * v2) / d

In [ ]:
drc

In [ ]:
drc_max = drc.subs(phi[0], Pi+phi[1])

In [ ]:
drc_max

In [ ]:
drc_mag = drc_max.magnitude().simplify()

In [ ]:
drc_mag

total force

In [ ]:
f_soc_max = drc_mag * mag_max

In [ ]:
f_soc_max

neighborhood

In [ ]:
r, R, t, v_max = symbols("r, R, t, v_max", positive=True)

In [ ]:
vals = {
    tau_0: 3,
    v: 2 * v_max,
    r_tot: 2 * r,
    x: R - 2 * t * v_max
}

In [ ]:
vals

In [ ]:
f_neigh_max = -f_soc_max.subs(vals)

In [ ]:
f_neigh_max

In [ ]:
vals2 = {
    v_max: 1.0,
    r: 0.255,
    t: 20 * 0.01
}

In [ ]:
err = f_neigh_max.subs(vals2)

In [ ]:
err

In [ ]:
f = lambdify(R, err, "numpy")

In [ ]:
R2 = np.linspace(3, 7)
plt.plot(R2, f(R2))

neighsize

In [ ]:
def neigh_size(R, r, rho):
    return rho * (R / r) ** 2

In [ ]:
plt.plot(R2, neigh_size(R2, 0.255, 0.70))

# Contact Force

$$
\begin{split}\mathbf{f}^{c} = - h \cdot \left(\mu \cdot \hat{\mathbf{n}} - \kappa \cdot (\mathbf{v} \cdot \hat{\mathbf{t}}) \hat{\mathbf{t}}\right) + c_{n} \cdot (\mathbf{v} \cdot \hat{\mathbf{n}}) \hat{\mathbf{n}} , \quad h < 0\end{split}
$$

# Time-to-collision for circle and three circle model

In [7]:
# Reference frame for creating vectors 
E = ReferenceFrame('E')  
# Relative position and velocity
x, v = symbols(("x_0:2", "v_0:2"), real=True)  
# Time to collision
tau = symbols("tau", positive=True)  
tau0 = symbols("tau0", positive=True)  
# Major and minor axes of the ellipse
r, r_t, r_s, r_ts = symbols("r r_t r_s r_ts", positive=True)
# Body angle of the agent
phi = symbols("varphi", real=True)

In [8]:
# Vector pointing to direction of agents body angle
tangent = -sin(phi) * E.x + cos(phi) * E.y  # Normal vector
tangent

- sin(varphi) e_x + cos(varphi) e_y

In [9]:
x2 = x[0] * E.x + x[1] * E.y

In [10]:
v2 = v[0] * E.x + v[1] * E.y

In [11]:
a = dot(v2, v2)

In [12]:
b = -dot(x2, v2)

In [13]:
c = dot(x2, x2) - r**2

In [14]:
d = sqrt(b**2 - a * c)

In [15]:
tau = (b - d) / a

In [16]:
tau

                    ______________________________________________________
                   ╱   ⎛  2     2⎞ ⎛   2     2     2⎞                   2 
-v₀⋅x₀ - v₁⋅x₁ - ╲╱  - ⎝v₀  + v₁ ⎠⋅⎝- r  + x₀  + x₁ ⎠ + (-v₀⋅x₀ - v₁⋅x₁)  
──────────────────────────────────────────────────────────────────────────
                                  2     2                                 
                                v₀  + v₁                                  

In [17]:
x2 + r_ts * tangent

(-rₜₛ⋅sin(varphi) + x₀) e_x + (rₜₛ⋅cos(varphi) + x₁) e_y

In [18]:
(_.magnitude()**2).expand().simplify()

   2                                                   2     2
rₜₛ  - 2⋅rₜₛ⋅x₀⋅sin(varphi) + 2⋅rₜₛ⋅x₁⋅cos(varphi) + x₀  + x₁ 

In [22]:
diff(b**2, x[0])

-2⋅v₀⋅(-v₀⋅x₀ - v₁⋅x₁)

# Time-to-collision for ellipse

In [72]:
# Reference frame for creating vectors 
E = ReferenceFrame('E')  
# Relative position and velocity
x, v = symbols(("x_0:2", "v_0:2"), real=True)  
# Time to collision
tau = symbols("tau", positive=True)  
# Major and minor axes of the ellipse
r, r_t = symbols("r r_t", positive=True)
# Body angle of the agent
phi = symbols("varphi", real=True)

In [73]:
# Vector pointing to direction of agents body angle
normal = cos(phi) * E.x + sin(phi) * E.y  # Normal vector
normal

cos(varphi) e_x + sin(varphi) e_y

In [74]:
# Center of mass of the agent through time (tau)
center = x[0] * E.x + x[1] * E.y + tau * (v[0] * E.x + v[1] * E.y)
center

(τ⋅v₀ + x₀) e_x + (τ⋅v₁ + x₁) e_y

In [75]:
# Agents radius vector through time at the direction of relative position
radius = r_t * dot(center, normal) * E.x + \
         r * cross(center, normal).magnitude() * E.y
radius

rₜ⋅((τ⋅v₀ + x₀)⋅cos(varphi) + (τ⋅v₁ + x₁)⋅sin(varphi)) e_x + r⋅│(τ⋅v₀ + x₀)⋅sin(varphi) - (τ⋅v₁ + x₁)⋅cos(varphi)│ e_y

In [76]:
# Actual radius is the magnitude of the radius vector
radius2 = radius.magnitude()**2
radius2

 2                                                    2     2                 
r ⋅((τ⋅v₀ + x₀)⋅sin(varphi) - (τ⋅v₁ + x₁)⋅cos(varphi))  + rₜ ⋅((τ⋅v₀ + x₀)⋅cos

                                   2
(varphi) + (τ⋅v₁ + x₁)⋅sin(varphi)) 

In [77]:
radius2.expand().collect(tau)

 2   2    2              2                                  2   2    2        
r ⋅x₀ ⋅sin (varphi) - 2⋅r ⋅x₀⋅x₁⋅sin(varphi)⋅cos(varphi) + r ⋅x₁ ⋅cos (varphi)

     2   2    2               2                                   2   2    2  
 + rₜ ⋅x₀ ⋅cos (varphi) + 2⋅rₜ ⋅x₀⋅x₁⋅sin(varphi)⋅cos(varphi) + rₜ ⋅x₁ ⋅sin (v

          2 ⎛ 2   2    2              2                                  2   2
arphi) + τ ⋅⎝r ⋅v₀ ⋅sin (varphi) - 2⋅r ⋅v₀⋅v₁⋅sin(varphi)⋅cos(varphi) + r ⋅v₁ 

    2             2   2    2               2                                  
⋅cos (varphi) + rₜ ⋅v₀ ⋅cos (varphi) + 2⋅rₜ ⋅v₀⋅v₁⋅sin(varphi)⋅cos(varphi) + r

 2   2    2        ⎞     ⎛   2          2              2                      
ₜ ⋅v₁ ⋅sin (varphi)⎠ + τ⋅⎝2⋅r ⋅v₀⋅x₀⋅sin (varphi) - 2⋅r ⋅v₀⋅x₁⋅sin(varphi)⋅cos

              2                                    2          2               
(varphi) - 2⋅r ⋅v₁⋅x₀⋅sin(varphi)⋅cos(varphi) + 2⋅r ⋅v₁⋅x₁⋅cos (varphi) + 2⋅rₜ

2          2               2                  

In [78]:
cse(_, optimizations='basic')  # Common sub-expression

⎛⎡⎛      2⎞  ⎛     2⎞                     ⎛      2⎞               ⎛      2⎞   
⎝⎣⎝x₀, x₀ ⎠, ⎝x₁, r ⎠, (x₂, sin(varphi)), ⎝x₃, x₂ ⎠, (x₄, x₁⋅x₃), ⎝x₅, x₁ ⎠, (

                  ⎛      2⎞               ⎛      2⎞                           
x₆, cos(varphi)), ⎝x₇, x₆ ⎠, (x₈, x₁⋅x₇), ⎝x₉, rₜ ⎠, (x₁₀, x₇⋅x₉), (x₁₁, x₃⋅x₉

                                                       ⎛       2⎞  ⎛       2⎞ 
), (x₁₂, x₀⋅x₁), (x₁₃, 2⋅x₁⋅x₂⋅x₆), (x₁₄, 2⋅x₂⋅x₆⋅x₉), ⎝x₁₅, v₀ ⎠, ⎝x₁₆, v₁ ⎠,

                                                                              
 (x₁₇, v₀⋅v₁), (x₁₈, v₀⋅x₀), (x₁₉, v₁⋅x₁), (x₂₀, v₀⋅x₁), (x₂₁, x₂⋅x₆⋅x₉), (x₂₂

                         ⎤  ⎡ 2                                               
, v₁⋅x₀), (x₂₃, x₁⋅x₂⋅x₆)⎦, ⎣τ ⋅(x₁₀⋅x₁₅ + x₁₁⋅x₁₆ - x₁₃⋅x₁₇ + x₁₄⋅x₁₇ + x₁₅⋅x

                                                                              
₄ + x₁₆⋅x₈) + 2⋅τ⋅(x₁₀⋅x₁₈ + x₁₁⋅x₁₉ + x₁₈⋅x₄ + x₁₉⋅x₈ + x₂₀⋅x₂₁ - x₂₀⋅x₂₃ + x

                                              

Approximating radius

In [79]:
c = symbols("c_0:3", real=True)

In [80]:
# Radius can be represented in form
ellipse = sqrt(c[0] + c[1] * tau + c[2] * tau**2)
ellipse

   ___________________
  ╱                 2 
╲╱  c₀ + c₁⋅τ + c₂⋅τ  

In [81]:
ellipse.diff(tau)

      c₁              
      ── + c₂⋅τ       
      2               
──────────────────────
   ___________________
  ╱                 2 
╲╱  c₀ + c₁⋅τ + c₂⋅τ  

In [82]:
ellipse.diff(tau, 2)

                      2   
         (c₁ + 2⋅c₂⋅τ)    
c₂ - ─────────────────────
       ⎛                2⎞
     4⋅⎝c₀ + c₁⋅τ + c₂⋅τ ⎠
──────────────────────────
     ___________________  
    ╱                 2   
  ╲╱  c₀ + c₁⋅τ + c₂⋅τ    